In [ ]:
# @title Get Video
!wget -P video/ "https://ml-hiring.fringecore.sh/capture_the_scene/video.mp4"

--2025-07-04 14:31:16--  https://ml-hiring.fringecore.sh/capture_the_scene/video.mp4
Resolving ml-hiring.fringecore.sh (ml-hiring.fringecore.sh)... 104.21.75.216, 172.67.182.61, 2606:4700:3037::ac43:b63d, ...
Connecting to ml-hiring.fringecore.sh (ml-hiring.fringecore.sh)|104.21.75.216|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3700355 (3.5M) [video/mp4]
Saving to: ‘video/video.mp4.3’

video.mp4.3         100%[===================>]   3.53M  --.-KB/s    in 0.09s   

2025-07-04 14:31:17 (39.8 MB/s) - ‘video/video.mp4.3’ saved [3700355/3700355]



# Function to Implement

In [ ]:
# The EVL pano code gets terminated while calling process_video_for_pano(video_file_path)
# For the access of insufficient RAM
# Although frames limit is 30 and size is  320x180
# Run this in premium Colab or in local machine for the output

import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

def process_video_for_pano(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return None

    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    print(f"Video properties: {width}x{height}, {fps} FPS, {total_frames} frames")

    # Sampling interval
    frame_interval = max(1, fps // 2)  # Sample 2 frames per second

    MAX_FRAMES = 30
    RESIZE_WIDTH = 320

    frames = []
    frame_count = 0

    print("Extracting frames...")
    while True:
        ret, frame = cap.read()
        if not ret or len(frames) >= MAX_FRAMES:
            break

        if frame_count % frame_interval == 0:
            frame = cv2.resize(frame, (RESIZE_WIDTH, int(frame.shape[0] * RESIZE_WIDTH / frame.shape[1])))
            frames.append(frame)
            if len(frames) % 10 == 0:
                print(f"Extracted {len(frames)} frames...")

        frame_count += 1

    cap.release()

    if len(frames) < 2:
        print("Error: Not enough frames extracted")
        return None

    print(f"Total frames extracted: {len(frames)}")

    # Initialize panorama
    panorama = frames[0].copy()

    # Feature matcher
    sift = cv2.SIFT_create()
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    print("Creating panorama...")
    successful_stitches = 0

    for i in range(1, len(frames)):
        current_frame = frames[i]

        try:
            kp1, des1 = sift.detectAndCompute(panorama, None)
            kp2, des2 = sift.detectAndCompute(current_frame, None)

            if des1 is None or des2 is None:
                continue

            matches = flann.knnMatch(des1, des2, k=2)

            good_matches = []
            for match_pair in matches:
                if len(match_pair) == 2:
                    m, n = match_pair
                    if m.distance < 0.7 * n.distance:
                        good_matches.append(m)

            if len(good_matches) < 10:
                continue

            src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
            dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

            H, mask = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)
            if H is None:
                continue

            h1, w1 = panorama.shape[:2]
            h2, w2 = current_frame.shape[:2]

            corners = np.float32([[0, 0], [w2, 0], [w2, h2], [0, h2]]).reshape(-1, 1, 2)
            transformed_corners = cv2.perspectiveTransform(corners, H)

            all_corners = np.concatenate([[[0, 0], [w1, 0], [w1, h1], [0, h1]], transformed_corners.reshape(-1, 2)])
            x_min, y_min = np.int32(all_corners.min(axis=0).ravel())
            x_max, y_max = np.int32(all_corners.max(axis=0).ravel())

            translation = np.array([[1, 0, -x_min], [0, 1, -y_min], [0, 0, 1]], dtype=np.float32)

            warped = cv2.warpPerspective(current_frame, translation @ H, (x_max - x_min, y_max - y_min))
            warped_panorama = cv2.warpPerspective(panorama, translation, (x_max - x_min, y_max - y_min))

            mask = (warped > 0).astype(np.uint8)
            result = np.where(mask, warped, warped_panorama)

            panorama = result
            successful_stitches += 1

            if i % 10 == 0:
                print(f"Processed {i}/{len(frames)} frames, successful stitches: {successful_stitches}")

        except Exception as e:
            print(f"Error processing frame {i}: {str(e)}")
            continue

    print(f"Panorama creation completed. Total successful stitches: {successful_stitches}")

    output_dir = "video"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    path_to_pano = os.path.join(output_dir, "panorama.jpg")
    cv2.imwrite(path_to_pano, panorama)

    plt.figure(figsize=(20, 6))
    panorama_rgb = cv2.cvtColor(panorama, cv2.COLOR_BGR2RGB)
    plt.imshow(panorama_rgb)
    plt.title("Video Panorama")
    plt.axis('off')
    plt.tight_layout()
    plt.show()

    print(f"Panorama saved to: {path_to_pano}")
    print(f"Panorama dimensions: {panorama.shape[1]}x{panorama.shape[0]}")


  # path_to_pano= None
    return path_to_pano


# Eval

In [ ]:
# @title Get Pano

video_file_path = '/content/video/video.mp4'
process_video_for_pano(video_file_path)


Video properties: 720x1280, 30 FPS, 255 frames
Extracting frames...
Extracted 10 frames...
Total frames extracted: 17
Creating panorama...
Processed 10/17 frames, successful stitches: 10
